In [2]:
import os
import sys
import numpy as np
import numpy.random as nrand
import pandas as pd
import pickle
import copy
import math
import matplotlib.pyplot as plt
from itertools import combinations
from matplotlib.ticker import FormatStrFormatter
from scipy.optimize import curve_fit
import itertools
from sklearn.linear_model import LinearRegression
import uncertainties as unc
import uncertainties.unumpy as unp
from multiprocessing import Pool
from lmfit import minimize, Parameters, fit_report
currentdir = os.path.abspath('')
targetdir = os.path.dirname(currentdir)+'/utils'
sys.path.append(targetdir)
from utils import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Define sigmoid and other mathematical modles for fitting

# Holling type III response model
def HT3(params, x, y):
    a = params['a']
    b = params['b']
    c = params['c']
    prediction = a*x**2/(a*b*x**2+1)+c
    return prediction-y

# Algebraic1 model
def Algebraic1(params, x, y):
    a = params['a']
    b = params['b']
    c = params['c']    
    prediction = a*x**3/(a*b*x**3+1)+c
    return prediction-y

# Algebraic2 model
def Algebraic2(params, x, y):
    a = params['a']
    b = params['b']
    c = params['c']
    d = params['d']
    prediction = a*(x-c)/((x-c)**2+b)**0.5+d
    return prediction-y

# Algebraic3 model
def Algebraic3(params, x, y):
    a = params['a']
    b = params['b']
    c = params['c']
    d = params['d']
    prediction = a*(x-c)/(abs(x-c)+b)+d
    return prediction-y

# Linear model
def Linear(params, x, y):
    a = params['a']
    b = params['b']
    prediction = a*x+b
    return prediction-y

# Quadratic model
def Quadratic(params, x, y):
    a = params['a']
    b = params['b']
    c = params['c']
    prediction = a*x**2+b*x+c
    return prediction-y

# Cubic model
def Cubic(params, x, y):
    a = params['a']
    b = params['b']
    c = params['c']
    d = params['d']
    prediction = a*x**3+b*x**2+c*x+d
    return prediction-y

# Logistic model
def Logistic(params, x, y):
    a = params['a']
    b = params['b']
    c = params['c']
    d = params['d']
    prediction = a/(1+np.e**(-b*(x-c)))+d
    return prediction-y

# SSE of model from mean
def SSE(popt,ruggedness_dict,rep):
    return sum((func(np.array([1/np.sqrt(i) for i in range(1,rep+1)]), *popt)-\
                np.array([np.mean(ruggedness_dict[i]) for i in range(1,rep+1)]))**2)

In [10]:
def fit_raw(ruggedness_dict,rep,method):
    """
    # Fitting sigmoid and other mathematical models to the averaged data points.
    """
    func = method_dict[method]
    params = Parameters()
    xdata = np.array([1/np.sqrt(i) for i in range(1,rep+1)])
    ydata = np.array([np.mean(ruggedness_dict[i]) for i in range(1,rep+1)])
    # Prepare model for fitting
    if method == 'Linear':
        params.add('a', value=1,min=0)
        params.add('b', value=1)
    elif method in ['HT3','Algebraic1']:
        params.add('a', value=1,min=0)
        params.add('b', value=1,min=0)
        params.add('c', value=1,min=0)
    elif method == 'Quadratic':
        params.add('a', value=1)
        params.add('b', value=1)
        params.add('c', value=1)
    elif method ==  'Cubic':
        params.add('a', value=1)
        params.add('b', value=-1)
        params.add('c', value=1)
        params.add('d', value=1)
    elif method in ['Algebraic2','Algebraic3']:
        params.add('a', value=ydata.max(),min=0,max=2*ydata.max()+1)
        params.add('b', value=1,min=0)
        params.add('c', value=1)
        params.add('d', value=0)
    elif method == 'Logistic':
        params.add('a', value=ydata.max(),min=0,max=2*ydata.max()+1)
        params.add('b', value=1,min=0)
        params.add('c', value=0.5,min=0)
        params.add('d', value=0)
    
    # Try two fitting method: 'leastsq' and 'least_squares' and use the results with errorbars.
    result1 = minimize(func, params, args=(xdata, ydata),method='leastsq')
    result = result1
    if not result1.errorbars:
        result2 = minimize(func, params, args=(xdata, ydata),method='least_squares')
        if result2.errorbars:
            result = result2
    return result #popt,sse

In [5]:
# Specify global lists
N_list = [5,10,15]
model_list = ['NK','RMF','Polynomial']
metric_list = ['N_max','epi','r_s','open_ratio','E','gamma','adptwalk_steps','adptwalk_probs']
method_list = ['HT3','Algebraic1','Algebraic2','Algebraic3','Linear','Quadratic','Cubic','Logistic']
method_dict= {'HT3':HT3,'Algebraic1':Algebraic1,'Algebraic2':Algebraic2,'Algebraic3':Algebraic3,
              'Linear':Linear,'Quadratic':Quadratic,'Cubic':Cubic,'Logistic':Logistic}

In [6]:
# Load and pre-process raw FL data
all_df = pd.DataFrame()
for N in N_list:
    for model in model_list:
        for metric in metric_list:
            with open(f'./raw_data/{model}{N}_{metric}_raw.pkl', 'rb') as f:
                res_dict = pickle.load(f)
            res_dict = pd.DataFrame(res_dict)
            all_df = pd.concat([all_df,res_dict],ignore_index=True)


In [7]:
# Convert ruggedness to a correct scale for open_ratio, gamma, adptwalk_probs, adptwalk_steps
raw_dict_list = []
for idx,row in all_df.iterrows():
    if row['metric'] in ['gamma','adptwalk_probs','open_ratio']:
        if row['metric'] == 'open_ratio':
            all_df.loc[idx,'metric'] = 'blocked_ratio'
        else:
            all_df.loc[idx,'metric'] = f"1-{row['metric']}"
        all_df.loc[idx,'ground_truth'] = 1- row['ground_truth']
        raw_dict = {}
        for i in range(row['replication']):
            raw_dict[i+1] = [1-k for k in row['raw'][i+1]]
    elif row['metric'] == 'adptwalk_steps':
        all_df.loc[idx,'metric'] = f"1/{row['metric']}"
        all_df.loc[idx,'ground_truth'] = 1/row['ground_truth']
        raw_dict = {}
        for i in range(row['replication']):
            raw_dict[i+1] = [1/k for k in row['raw'][i+1]]
    else:
        raw_dict = row['raw']
    raw_dict_list.append(raw_dict)
all_df.loc[:,'raw'] = raw_dict_list

In [8]:
# Add averaged_ruggedness
averaged_ruggedness = []
for i,item in all_df.iterrows():
    averaged_ruggedness.append(item['raw'][int(item['replication'])][0])
all_df['averaged_ruggedness'] = averaged_ruggedness

In [9]:
# Pre-calculate convex_concave information for replication of 3.
convex_concave = []
for i,item in all_df.iterrows():
    if item['replication'] == 3:
        if np.mean(item['raw'][1]) + np.mean(item['raw'][3]) >= 2*np.mean(item['raw'][2]):
            convex_concave.append('concave')
        else:
            convex_concave.append('convex')
    else:
        convex_concave.append(None)
all_df['convex_concave'] = convex_concave

In [16]:
def worker(idx):
    """
    Worker function used for multithreading.
    idx: the index of the data showing where to start analysis.
    """
    # A: alpha, B: beta, C: delta
    res_dict = {'prediction':[],'prediction_std':[],'fitting_sse':[],'method':[],'A':[],'B':[],'C':[],'accept':[]}
    cv_cutoff = 0.5
    for i in range(idx*1300,(idx+1)*1300): 
        
        # Stop loop once reached the upper limit 31104
        if i >= 31104:
            break

        if idx == 0:
            print(i)
        
        raw_data = all_df.loc[i,'raw']
        rep = all_df.loc[i,'replication']
        groundt = all_df.loc[i,'ground_truth']
        averaged_ruggedness = all_df.loc[i,'averaged_ruggedness']
        convex_concave = all_df.loc[i,'convex_concave']
        
        for method in method_list:
            # if there is only 3 replicates but need to fit model with 4 or 
            # more than 4 parameters, skip this method.
            if method in ['Algebraic2','Algebraic3', 'Cubic', 'Logistic'] and rep == 3:
                continue
                
            # Fit model to data
            result = fit_raw(raw_data,rep,method)
            if result.errorbars:
                pcov = result.covar
            popt = list(result.params.valuesdict().values())
            
            # First assume the result is resonable to accept
            accept = True
            
            try:
                # Don't calculate prediction std, SEE, and covariance matrix if there is no errorbar estimation
                if method == 'Linear' and result.errorbars:
                    a,b = unc.correlated_values(popt,pcov)
                    y0 = method_dict[method]({'a':a, 'b':b},0,0)
                    # Get extrapolated true ruggedness
                    y0_nom = float(unp.nominal_values(y0))
                    # Get std of extrapolated true ruggedness
                    y0_std = float(unp.std_devs(y0))
                    # Get SSE
                    sse = result.chisqr
                    # Don't accept the prediction if standard deviation of the prediction
                    # is greater than half of averaged ruggendess.
                    if np.abs(y0_std/averaged_ruggedness) > cv_cutoff :
                        accept = False
                elif method in ['HT3','Algebraic1', 'Quadratic'] and result.errorbars:
                    a,b,c = unc.correlated_values(popt,pcov)
                    y0 = method_dict[method]({'a':a, 'b':b, 'c':c},0,0)
                    y0_nom = float(unp.nominal_values(y0))
                    y0_std = float(unp.std_devs(y0))
                    sse = result.chisqr
                    if np.abs(y0_std/averaged_ruggedness) > cv_cutoff :
                        accept = False
                elif method in ['Algebraic2','Algebraic3', 'Cubic', 'Logistic'] and result.errorbars:
                    a,b,c,d = unc.correlated_values(popt,pcov)
                    y0 = method_dict[method]({'a':a, 'b':b, 'c':c, 'd':d},0,0)
                    y0_nom = float(unp.nominal_values(y0))
                    y0_std = float(unp.std_devs(y0))
                    sse = result.chisqr
                    if np.abs(y0_std/averaged_ruggedness) > cv_cutoff :
                        accept = False
                else:
                    # If there are no errorbar estimation. 
                    # Accept the prediction if the left most data point is located
                    # in the convex part of the curve.
                    y0_nom = method_dict[method](result.params,0,0)
                    y0_std = None
                    sse = None
                    
                    # If there are 3 replicates
                    if rep == 3:
                        if method == 'Linear': pass
                        a,b,c = popt
                        if method == 'HT3' and np.sqrt(1/(3*a*b)) < 1/np.sqrt(rep):
                            accept = False
                        elif method == 'Algebraic1' and np.cbrt(1/(2*a*b)) < 1/np.sqrt(rep):
                            accept = False
                        elif method == 'Quadratic' and a<=0:
                            accept = False
                            
                    # If there are 4 replicates
                    elif rep ==4:
                        if method not in ['Algebraic2', 'Algebraic3', 'Logistic']:
                            accept = False
                        else:
                            a,b,c,d = popt
                        if method in ['Algebraic2', 'Algebraic3', 'Logistic'] and c < 1/np.sqrt(rep):
                            accept = False
                        elif method == 'Cubic' and -b/(3*a) < 1/np.sqrt(rep):
                            accept = False
            
            # If OverflowError happens in fitting process,
            # We don't accept the prediction
            except OverflowError:
                print('OverflowError')
                y0_nom = 0
                y0_std = 0
                sse = 0
                accept = False
                pass
            
            # Record fitting and prediction results
            for key in all_df.keys():
                if key not in res_dict:
                    res_dict[key] = []
                res_dict[key].append(all_df.loc[i,key])
            res_dict['prediction'].append(y0_nom)
            res_dict['prediction_std'].append(y0_std)
            res_dict['fitting_sse'].append(sse)
            res_dict['method'].append(method)
            #res_dict['popt'].append(popt)
            #res_dict['pcov'].append(pcov)
            res_dict[f'A'].append(np.mean(all_df.loc[i,"raw"][1]) - groundt)
            res_dict[f'B'].append(all_df.loc[i,"raw"][rep][0] - groundt)
            res_dict[f'C'].append(y0_nom - groundt)
            res_dict['accept'].append(accept)
                
    return pd.DataFrame(res_dict)
    
    

In [ ]:
# Multithreading
with Pool(24) as p:
    res_df = p.map(worker, range(24))
res_df = pd.concat(res_df,ignore_index=True)

In [19]:
# Uncomment only if you want to overwrite extrapolation_result.pkl.
# with open('extrapolation_model_selection_result.pkl','wb') as f:
#     pickle.dump(res_df,f)